In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
data = pd.read_csv('Data.txt')
data.head()

,7.74108464
0,7.762000
1,7.788000
2,7.808400
3,7.823564
4,7.812000


In [21]:
# Rename a column

data.rename(columns={'7.74108464':'Seq_num'},inplace=True)
data

,Seq_num
0,7.762000
1,7.788000
2,7.808400
3,7.823564
4,7.812000
...,...
4994,7.738000
4995,7.738000
4996,7.740000
4997,7.739548


In [22]:
# print the data now.
data.tail(10)

,Seq_num
4989,7.730600
4990,7.734000
4991,7.738000
4992,7.740793
4993,7.738000
4994,7.738000
4995,7.738000
4996,7.740000
4997,7.739548
4998,7.742000


In [23]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.layers import Flatten
import math
from sklearn.metrics import mean_squared_error

In [24]:
# prepairing independent and dependent data

def prepare_data(time_ser_data,n_features):
    x,y=[],[]
    for i in range(len(time_ser_data)):
        # find the end of this pattern

        end_pattern = i+n_features
        # check if we are beyond the sequenec
        if end_pattern >len(time_ser_data)-1:
            break

        # gather input and output part of the pattern
        seq_x,seq_y = time_ser_data[i:end_pattern],time_ser_data[end_pattern]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x),np.array(y)

In [25]:
# define input sequence

time_ser_data =data.iloc[:,0]
time_ser_data.shape

(4999,)

In [26]:
# choose a  number of time steps

n_steps= 5

# split into samples
x , y = prepare_data(time_ser_data,n_steps)

In [27]:
print(x) 
x.shape

[[7.762      7.788      7.8084     7.82356438 7.812     ]
 [7.788      7.8084     7.82356438 7.812      7.82      ]
 [7.8084     7.82356438 7.812      7.82       7.8288    ]
 ...
 [7.738      7.74079268 7.738      7.738      7.738     ]
 [7.74079268 7.738      7.738      7.738      7.74      ]
 [7.738      7.738      7.738      7.74       7.73954768]]


(4994, 5)

In [28]:
# convert sample feature [samples, timesteps] to 3D [samples,timesteps,n_features]

n_feature = 1
x = x.reshape((x.shape[0],x.shape[1],n_feature))
x.shape 

(4994, 5, 1)

In [29]:
# implement LSTM model.

model = Sequential()
model.add(LSTM(50,activation='relu',return_sequences=True, input_shape=(n_steps,n_feature )))
model.add(LSTM(50,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam',loss = 'mse')

In [30]:
# fit model

model.fit(x,y ,epochs=200 , verbose=1)

Epoch 1/200
157/157 [==============================] - 5s 10ms/step - loss: 9.1159
Epoch 2/200
157/157 [==============================] - 2s 11ms/step - loss: 0.0555
Epoch 3/200
157/157 [==============================] - 2s 12ms/step - loss: 0.0397
Epoch 4/200
157/157 [==============================] - 2s 12ms/step - loss: 0.0226
Epoch 5/200
157/157 [==============================] - 2s 12ms/step - loss: 0.0059
Epoch 6/200
157/157 [==============================] - 2s 12ms/step - loss: 8.6461e-04
Epoch 7/200
157/157 [==============================] - 2s 11ms/step - loss: 8.0802e-04
Epoch 8/200
157/157 [==============================] - 2s 11ms/step - loss: 7.3570e-04
Epoch 9/200
157/157 [==============================] - 2s 11ms/step - loss: 7.5169e-04
Epoch 10/200
157/157 [==============================] - 2s 11ms/step - loss: 6.5631e-04
Epoch 11/200
157/157 [==============================] - 2s 12ms/step - loss: 8.1511e-04
Epoch 12/200
157/157 [==============================] - 2s 11

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 5, 50)             10400     
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [34]:
# predict next 4 days

x_input= np.array([7.738000,7.738000,7.740000,7.739548,7.742000])

list_x= list(x_input)
lst_output = []
i = 0
while(i<4):
    if(len(list_x)>5):
        x_input=np.array(list_x[1:])
        print('{} day input {}'.format(i,x_input))

        # print x_input
        x_input= x_input.reshape((1,n_steps,n_feature))

        # print y_pred

        y_pred = model.predict(x_input,verbose = 0)
        print('{} day output {}'.format(i,y_pred))
        list_x.append(y_pred[0][0])
        list_x = list_x[1:]

        lst_output.append(y_pred[0][0])
        i = i+1
    else:
        x_input= x_input.reshape((1,n_steps,n_feature))
        y_pred = model.predict(x_input,verbose = 0)
        print(y_pred[0])
        list_x.append(y_pred[0][0])
        lst_output.append(y_pred[0][0])
        i = i+1
    print(lst_output)
        


[7.746554]
[7.746554]
1 day input [7.738     7.74      7.739548  7.742     7.7465539]
1 day output [[7.7483087]]
[7.746554, 7.7483087]
2 day input [7.74       7.739548   7.742      7.7465539  7.74830866]
2 day output [[7.7505655]]
[7.746554, 7.7483087, 7.7505655]
3 day input [7.739548   7.742      7.7465539  7.74830866 7.75056553]
3 day output [[7.752796]]
[7.746554, 7.7483087, 7.7505655, 7.752796]
